<a href="https://colab.research.google.com/github/JALVARADORUIZ/Magister_BigData_UB/blob/main/M2_AG1_GRUPO14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FUNDAMENTOS DE ESTADÍSTICA**
PADAWANS:
- PAULA VALLEJO VARGAS
- JOSE ARMANDO ALVARADO RUIZ

# **Introducción al cálculo de probabilidades**



**Entrega: M2_AG1_CORONAVIRUS**

# Descripción de la tarea

Tenéis un dataset (worlddometer_covid_ue.csv) con los datos a nivel europeo del Covid-19. En el momento del estudio, se detecta la siguiente situación:

Hay 6 países europeos con el mayor número de casos por Covid-19 que son Rusia, España, UK, Francia, Italia y Alemania con, aproximadamente, 250.000 casos.

Por lo que sabemos, se han adoptado algunas medidas similares de contención (confinamiento, mascarilla, geles, etc.) aunque no se han aplicado en las mismas fechas (por ejemplo, Italia comenzó 2 semanas antes que España y, en UK, postergaron el confinamiento en más de 15 días con respecto a España).

El número de habitantes por población es similar, quitando a Rusia que tiene el triple de personas que en España. Esto hace que descartemos a Rusia del análisis comparativo. En este punto, cabría la opción de plantearse añadir a Ucrania que tiene un número similar de habitantes que España.

Con los datos facilitados sobre el total de casos confirmados con Covid-19 (suponemos que con PCR) y sobre el total de fallecimientos acontecidos, te pedimos que halles:

- La probabilidad de sobrevivir en cualquiera de los seis países europeos seleccionados para el estudio.
- Si seleccionamos una de las personas fallecidas, ¿cuál es la probabilidad de que residiera en España? ¿Y en el resto de países?

Justifica y comenta los resultados obtenidos.

CONSIDERACIONES

Fuente de la información.

Se supondrá que los datos de fallecimiento se han contabilizado exactamente igual en los países seleccionados para el análisis.

Los datos son oficiales, con fecha 29-08-2020, pero esto no quiere decir que se puedan extraer conclusiones definitivas, mejor dicho, hay que considerar más variables de las que se tienen en cuenta en este análisis como, por ejemplo, la distribución de la edad en la población, la forma de considerar los fallecimientos por Covid-19 (si son con PCR o no todos los que figuran), las medidas adoptadas, etc.

Justifica adecuadamente tus respuestas.

# Carga y análisis de la Data

IMPORTACIÓN DE LIBRERÍAS

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import statsmodels.api as sm
import seaborn as sns
import math
import pandas.util.testing as tm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
%matplotlib inline

# Carga de Datos y visualización general

url = 'https://raw.githubusercontent.com/md-lorente/data/master/worlddometer_covid_ue.csv'
#df = pd.read_csv(url, sep=";", decimal=",")
df = pd.read_csv(url, sep=';',decimal = '.', thousands= ',')
df = df.infer_objects()
df.head(10)

,Num,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious Critical,Total Cases per 1M pop,Deaths per 1M pop,Total Tests,Tests per 1M pop,Population
0,0,Europe,3543710,10493.0,207347.0,149.0,2063335.0,1273028.0,5619.0,NaN,NaN,NaN,NaN,NaN
1,1,Russia,990326,498.0,17093.0,68.0,806982.0,166251.0,23.0,6786.0,117.0,36400000.0,249409.0,145944843.0
2,2,Spain,455621,NaN,29011.0,NaN,NaN,NaN,751.0,9744.0,620.0,8517446.0,182161.0,46757783.0
3,3,UK,332752,NaN,41498.0,NaN,NaN,NaN,60.0,4897.0,611.0,16273209.0,239508.0,67944393.0
4,4,France,272530,NaN,30602.0,NaN,86177.0,155751.0,387.0,4174.0,469.0,6175223.0,94571.0,65297356.0
5,5,Italy,266853,NaN,35473.0,NaN,208224.0,23156.0,79.0,4415.0,587.0,8509618.0,140778.0,60447003.0
6,6,Germany,242825,NaN,9363.0,NaN,217484.0,15978.0,245.0,2897.0,112.0,11208091.0,133703.0,83828051.0
7,7,Ukraine,119074,2096.0,2527.0,35.0,56734.0,59813.0,177.0,2725.0,58.0,1536171.0,35161.0,43689434.0
8,8,Romania,86785,952.0,3578.0,39.0,37665.0,45542.0,492.0,4516.0,186.0,1795633.0,93445.0,19216003.0
9,9,Belgium,84599,647.0,9891.0,5.0,18403.0,56305.0,72.0,7294.0,853.0,2261043.0,194953.0,11597901.0


In [57]:
df.columns

Index(['Num', 'Country', 'Total Cases', 'New Cases', 'Total Deaths',
       'New Deaths', 'Total Recovered', 'Active Cases', 'Serious Critical',
       'Total Cases per 1M pop', 'Deaths per 1M pop', 'Total Tests',
       'Tests per 1M pop', 'Population '],
      dtype='object')

In [58]:
df.dtypes

Num                         int64
Country                    object
Total Cases                 int64
New Cases                 float64
Total Deaths              float64
New Deaths                float64
Total Recovered           float64
Active Cases              float64
Serious Critical          float64
Total Cases per 1M pop    float64
Deaths per 1M pop         float64
Total Tests               float64
Tests per 1M pop          float64
Population                float64
dtype: object

# Seleccion Universo

El universo está formado por estos 6 países: 

- España
- UK
- Francia
- Italia
- Alemania
- Ucrania.

In [59]:
# Seleccionamos a los 6 paises que serán nuestro Universo

df_universo = df.loc[2:7]

In [61]:
df_universo

,Num,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious Critical,Total Cases per 1M pop,Deaths per 1M pop,Total Tests,Tests per 1M pop,Population
2,2,Spain,455621,NaN,29011.0,NaN,NaN,NaN,751.0,9744.0,620.0,8517446.0,182161.0,46757783.0
3,3,UK,332752,NaN,41498.0,NaN,NaN,NaN,60.0,4897.0,611.0,16273209.0,239508.0,67944393.0
4,4,France,272530,NaN,30602.0,NaN,86177.0,155751.0,387.0,4174.0,469.0,6175223.0,94571.0,65297356.0
5,5,Italy,266853,NaN,35473.0,NaN,208224.0,23156.0,79.0,4415.0,587.0,8509618.0,140778.0,60447003.0
6,6,Germany,242825,NaN,9363.0,NaN,217484.0,15978.0,245.0,2897.0,112.0,11208091.0,133703.0,83828051.0
7,7,Ukraine,119074,2096.0,2527.0,35.0,56734.0,59813.0,177.0,2725.0,58.0,1536171.0,35161.0,43689434.0


# **Probabilidad de Laplace**

**1. La probabilidad de sobrevivir en cualquiera de los seis países europeos seleccionados para el estudio.**

Considerando los datos disponibles, consideraremos que la Probabilidad de sobrevivir esta dada por:

        1 - Probabilidad de Morir

donde Probabilidad de Morir es *total_deaths/total_cases*, por lo tanto:

        Probabilidad de sobrevivir = 1 - (total_deaths/total_cases)

En este caso, estamos utilizando la Probabilidad a traves de Laplace, ya que bastaría con determinar la proporción entre el número de casos favorables de un suceso y el número de resultados posibles relativos al experimento, donde los casos favorables serían el total de muertes y el número posible es el total de casos, basado en los datos facilitados sobre el total de casos confirmados con Covid-19 (suponemos que con PCR) y sobre el total de fallecimientos acontecidos.

In [62]:
# Analicemos el caso de España:

prob_morir_sp = df_universo.loc[df_universo['Country'] =='Spain']['Total Deaths'] / df_universo.loc[df_universo['Country'] =='Spain']['Total Cases']
prob_sobrevivir_sp = 1 - prob_morir_sp.values[0]
print("La Probabilidad de Sobrevivir en España es de : ", round(prob_sobrevivir_sp,3))

La Probabilidad de Sobrevivir en España es de :  0.936


In [63]:
# Analicemos el caso de UK:

prob_morir_uk = df_universo.loc[df_universo['Country'] =='UK']['Total Deaths'] / df_universo.loc[df_universo['Country'] =='UK']['Total Cases']
prob_sobrevivir_uk = 1 - prob_morir_uk.values[0]
print("La Probabilidad de Sobrevivir en UK es de : ", round(prob_sobrevivir_uk,3))

La Probabilidad de Sobrevivir en UK es de :  0.875


In [64]:
# Analicemos el caso de Francia:

prob_morir_fr = df_universo.loc[df_universo['Country'] =='France']['Total Deaths'] / df_universo.loc[df_universo['Country'] =='France']['Total Cases']
prob_sobrevivir_fr = 1 - prob_morir_fr.values[0]
print("La Probabilidad de Sobrevivir en Francia es de : ", round(prob_sobrevivir_fr,3))

La Probabilidad de Sobrevivir en Francia es de :  0.888


In [65]:
# Analicemos el caso de Italia:

prob_morir_it = df_universo.loc[df_universo['Country'] =='Italy']['Total Deaths'] / df_universo.loc[df_universo['Country'] =='Italy']['Total Cases']
prob_sobrevivir_it = 1 - prob_morir_it.values[0]
print("La Probabilidad de Sobrevivir en Italia es de : ", round(prob_sobrevivir_it,3))

La Probabilidad de Sobrevivir en Italia es de :  0.867


In [66]:
# Analicemos el caso de Alemania:

prob_morir_de = df_universo.loc[df_universo['Country'] =='Germany']['Total Deaths'] / df_universo.loc[df_universo['Country'] =='Germany']['Total Cases']
prob_sobrevivir_de = 1 - prob_morir_de.values[0]
print("La Probabilidad de Sobrevivir en Alemania es de : ", round(prob_sobrevivir_de,3))

La Probabilidad de Sobrevivir en Alemania es de :  0.961


In [67]:
# Analicemos el caso de Ucrania:

prob_morir_ua = df_universo.loc[df_universo['Country'] =='Ukraine']['Total Deaths'] / df_universo.loc[df_universo['Country'] =='Ukraine']['Total Cases']
prob_sobrevivir_ua = 1 - prob_morir_ua.values[0]
print("La Probabilidad de Sobrevivir en Ucrania es de : ", round(prob_sobrevivir_ua,3))

La Probabilidad de Sobrevivir en Ucrania es de :  0.979


Analisis de Resultados

Podemos ver que la mayor probabilidad de sobrevivir al Coronavirus dentro de nuestro universo de 6 países seleccionados esta en Ucrania con un 97,9%, mientras que la menor probabilidad la tiene es Italia con un 86,7%.

A continuación un cuadro comparativo de nuestra universo:



# Teorema de la Probabilidad Total y Teorema de Bayes

Si seleccionamos una de las personas fallecidas, ¿cuál es la probabilidad de que residiera en España? ¿Y en el resto de países?

In [97]:
df2 = df_universo[['Country','Total Cases','Total Deaths']]

In [98]:
# Total de Casos universo 

total_casos_universo = df2['Total Cases'].sum()
df2['Porcentaje Casos'] = df2['Total Cases'] / total_casos_universo




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [99]:
df2

,Country,Total Cases,Total Deaths,Porcentaje Casos
2,Spain,455621,29011.0,0.269653
3,UK,332752,41498.0,0.196935
4,France,272530,30602.0,0.161293
5,Italy,266853,35473.0,0.157933
6,Germany,242825,9363.0,0.143713
7,Ukraine,119074,2527.0,0.070472


Para calcular la probabilidad de que una persona muera en el universo de seis países, P(M) se puede calcular por la propiedad de la probabilidad total, esto es:

$$ 
P(M) = \sum_{j=1}^6 P(A_{j}) P(M|A_{j}) =  P(A_{1}) \cdot P(M|A_{1}) + P(A_{2}) \cdot P(M|A_{2}) + \ldots + P(A_{6}) \cdot P(M|A_{6})
\\  $$

Donde A es cada uno de los países.

In [100]:
def probabilidad_total(df):
  prob_total = 0
  for index, row in df.iterrows():
    prob_total = prob_total + row['Porcentaje Casos']*(row['Total Deaths']/row['Total Cases'])
  return prob_total

probabilidad_total = probabilidad_total(df2)

probabilidad_total

0.08787237631350779

Ahora podemos utilizar el Teorema de Bayes para determinar la probabilidad de que esa persona muerta resida en España, utilizando esta formula:

$$ 
P(Spain|M) = \frac{ P(Spain) \cdot P(M|Spain) }{ Probabilidad total }
\\ \\
$$

In [83]:
def teorema_bayes(country, df, prob_total):
  numerador = df.loc[df['Country']==country]['Porcentaje Casos']*(df.loc[df['Country']==country]['Total Deaths']/df.loc[df['Country']==country]['Total Cases'])
  bayes = numerador/prob_total
  return bayes

print(teorema_bayes('Spain',df2,probabilidad_total).values[0])


0.1953944798415884


In [88]:
print(teorema_bayes('UK',df2,probabilidad_total).values[0])

0.27949674690518206


In [89]:
print(teorema_bayes('France',df2,probabilidad_total).values[0])

0.20611016070153695


In [90]:
print(teorema_bayes('Italy',df2,probabilidad_total).values[0])

0.23891725150531404


In [91]:
print(teorema_bayes('Germany',df2,probabilidad_total).values[0])

0.06306154612928862


In [92]:
print(teorema_bayes('Ukraine',df2,probabilidad_total).values[0])

0.017019814917089857
